# Using Kinematics from Reachy's URDF file

## Creating the Chain from the URDF

We use the [Ikpy](https://github.com/Phylliade/ikpy) library to create the kinematic chain from the URDF. You have to specify:

* the path to the URDF file
* the base element of your chain
* the translation vector for the end tip

In [ ]:
from ikpy.chain import Chain

urdf_file = '../dev/reachy/reachy.URDF'

reachy_kinematic_chain = Chain.from_urdf_file(urdf_file, base_elements=['base'], last_link_vector=[0, 0, -0.02409])

You can check the created chain:

In [ ]:
for link in reachy_kinematic_chain.links:
    print(link)

## Forward kinematic

You can then call the forward kinematic method by specifying all joint angle (in radians). You have to specify 9 joints in the case of Reachy: the 7 defined joints plus one virtual before and one virtual at the end.

For instance, when setting all joints to 0 except the elbow pitch at $\pi /2$:

In [ ]:
import numpy

target = reachy_kinematic_chain.forward_kinematics(joints=[0, 0, 0, 0, numpy.pi / 2, 0, 0, 0, 0])
print(target)

You can then extract the rotation matrix:

In [ ]:
rot = target[:3,:3]
print(rot)

And the translation vector:

In [ ]:
trans = target[:3, -1]
print(trans)

## Inverse kinematics

You can also use the inverse kinematics to find the joint angles from a target. The target must be passed as a 4x4 homogeneous matrix.

In [ ]:
rot = numpy.eye(3)
trans = [0.09, 0.28, -0.30]

target = numpy.eye(4)
target[:3, :3] = rot
target[:3, 3] = trans

In [ ]:
joints = reachy_kinematic_chain.inverse_kinematics(target)
print(joints)

You can also specify an initial position for the inverse kinematics optimisation. This will drastically speed up the process and help find better solutions.

In [ ]:
joints = reachy_kinematic_chain.inverse_kinematics(target, initial_position=[0, 0, 0, 0, numpy.pi/2, 0, 0, 0, 0])
joints

## Use it with reachy

In [ ]:
from reachy import Reachy

reachy = Reachy()

In [ ]:
def goto(joints):
    for m, j in zip(reachy.motors, joints[1:-1]):
        m.goal_position = numpy.rad2deg(j)

In [ ]:
for m in reachy.motors:
    m.moving_speed = 50

In [ ]:
goto(joints)

### You can find more information on Ikpy documentation: https://github.com/Phylliade/ikpy